In [3]:
!pip install tensorflow  opencv-python mediapipe sklearn matplotlib pillow

In [6]:
pip install pysocks scikit-learn  

  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.4 kB 435.7 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.4 kB 435.7 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.4 kB 196.9 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 321.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.2 MB 2.6 MB/s eta 0:00:04
   ---------------------------------------- 0.1/9.2 MB 2.6 MB/s eta 0:00:04
   ---------------------------------------- 0.1/9.

In [10]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 
current_directory = os.getcwd()
# Actions that we try to detect
# actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ','ඔ','ඕ','ක්','ග්','ජ්','ට්','ද්','ණ්','ත්','ඩ්','න්','ප්','බ්','ම්','ය්','ර්','ල්','ව්','ස්','හ්'])
actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ'])				
# Thirty videos worth of data
no_sequences = 30

# Each One Video  going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
sequences, labels = [], []
# loop each action
for action in actions:
    # Loop through the each action sequence(videos)
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # this array will hold all the frame in one video
        window = []
        # Loop though the each frame 30(Frame)
        for frame_num in range(sequence_length):
            # using np.load function load the saved np array from folder (path,which action,which video number,which frame number)
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # add loaded framee to window
            window.append(res)
        #    
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
# shape should be 90 videos,30 frame each,1353 key points in a frame
np.array(sequences).shape

(150, 30, 126)

In [15]:
np.array(labels).shape

(150,)

In [16]:
X = np.array(sequences)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [19]:
from tensorflow.keras.models import Sequential
# Make us able to action detection
from tensorflow.keras.layers import LSTM, Dense
# Make us able to monitor our modle during taining
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [26]:
# load Sequential api
model = Sequential()
# add 3 layers of lstm model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [28]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
5/5 [==============================] - 4s 27ms/step - loss: 1.6056 - categorical_accuracy: 0.1972
Epoch 2/2000
5/5 [==============================] - 0s 26ms/step - loss: 1.5828 - categorical_accuracy: 0.3803
Epoch 3/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.4835 - categorical_accuracy: 0.4296
Epoch 4/2000
5/5 [==============================] - 0s 26ms/step - loss: 1.3688 - categorical_accuracy: 0.4366
Epoch 5/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.2425 - categorical_accuracy: 0.4930
Epoch 6/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.2538 - categorical_accuracy: 0.3873
Epoch 7/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.1475 - categorical_accuracy: 0.4366
Epoch 8/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.9901 - categorical_accuracy: 0.4930
Epoch 9/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.8997 - categorical_accuracy: 0.5704
E

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            48896     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 5)                 165       
                                                                 
Total params: 203525 (795.02 KB)
Trainable params: 203

In [31]:
res = model.predict(X_test)

1/1 [==============================] - 1s 514ms/step


In [32]:
actions[np.argmax(res[4])]

'අ'

In [33]:
actions[np.argmax(y_test[4])]

'අ'

In [36]:
model.save('action.h5')